In [1]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from pyspark.sql.functions import *
import ConnectionConfig as cc
from pyspark.sql.types import *
cc.setupEnvironment()


In [2]:
#active session
spark = cc.startLocalCluster("DIM_VEHICLE",4)
spark.getActiveSession()

In [3]:
# EXTRACT
cc.set_connectionProfile("VeloBike")
print(cc.create_jdbc())

jdbc:postgresql://localhost:5433/velodb


In [4]:


vehicle_df = spark.read.format("jdbc").option("driver" , "org.postgresql.Driver").option("url", cc.create_jdbc()).option("dbtable", "vehicles").option("user", cc.get_Property("username")).option("password", cc.get_Property("password"))    .load()

biketype_df = spark.read.format("jdbc").option("driver" , "org.postgresql.Driver").option("url", cc.create_jdbc()).option("dbtable", "bike_types").option("user", cc.get_Property("username")).option("password", cc.get_Property("password"))    .load()

bikelot_df = spark.read.format("jdbc").option("driver" , "org.postgresql.Driver").option("url", cc.create_jdbc()).option("dbtable", "bikelots").option("user", cc.get_Property("username")).option("password", cc.get_Property("password"))    .load()

# biketype_df.show(20)

biketype_df = biketype_df.withColumnRenamed("biketypeid", "biketype_id")
biketype_df.printSchema()

root
 |-- biketype_id: integer (nullable = true)
 |-- biketypedescription: string (nullable = true)



In [5]:
# TRANSFORM
bikelot_biketype_df = (bikelot_df.join(biketype_df,
                                      bikelot_df.biketypeid == biketype_df.biketype_id, "inner").select(col("bikelotid"),col("biketypeid"),col("biketypedescription")))

final_dim_vehicle = (vehicle_df.join(bikelot_biketype_df,
                                      vehicle_df.bikelotid == bikelot_biketype_df.bikelotid, "inner")
                      .select(col("biketypedescription"))
                      .distinct()
                      .withColumn("vehicle_SK", monotonically_increasing_id()+1))


final_dim_vehicle.show()


+-------------------+----------+
|biketypedescription|vehicle_SK|
+-------------------+----------+
|            Scooter|         1|
|          Velo Bike|         2|
|        Velo E-Bike|         3|
|               Step|         4|
+-------------------+----------+



In [6]:

# Write vehicle dimension to tables
final_dim_vehicle.write.format("delta").mode("overwrite").saveAsTable("dimVehicle")
final_dim_vehicle.repartition(1).write.format("parquet").mode("overwrite").saveAsTable("dimVehicle_pq")


In [7]:
spark.stop()